In [1]:
import os
import sys
import numpy as np
import ipympl
import matplotlib
#matplotlib.use('nbagg')
from matplotlib import style
#style.use('ggplot')
import matplotlib.pyplot as plt

import astropy.units as u
from astropy.io import fits
from astropy.table import Table

from mmtwfs.wfs import *
from mmtwfs.zernike import ZernikeVector
from mmtwfs.telescope import MMT

from camsrv.header import HEADER_MAP

from pathlib import Path

In [35]:
def get_header_info(keylist, fitsfile):
    info = []
    h = fits.open(fitsfile)
    hdr = h[0].header
    for k in keylist:
        if k in hdr:
            info.append(hdr[k])
        else:
            print(k)
            info.append(None)
    h.close()
    return info

def f5_model(key, el, t_oss):
    model = {}
    sin_el = np.sin(el * u.deg)
    cos_el = np.cos(el * u.deg)
    model['TRANSX'] = 787.7 - 507.6 * sin_el - 947.2 * cos_el + 0.0 * t_oss
    model['TRANSY'] = 1780.0 - 135.8 * sin_el + 1538.0 * cos_el + 0.0 * t_oss
    model['TRANSZ'] = 13608.5 + 671.1 * sin_el - 286.7 * cos_el - 13.1 * t_oss
    model['TILTX'] = 236.7 - 66.1 * sin_el + 119.6 * cos_el + 0.0 * t_oss
    model['TILTY'] = 70.5 + 20.5 * sin_el + 72.0 * cos_el + 0.0 * t_oss
    return model[key]

def bino_model(key, el, t_oss):
    model = {}
    sin_el = np.sin(el * u.deg)
    cos_el = np.cos(el * u.deg)
    model['TRANSX'] = 787.7 - 507.6 * sin_el - 947.2 * cos_el + 0.0 * t_oss
    model['TRANSY'] = 1780.0 - 135.8 * sin_el + 1538.0 * cos_el + 0.0 * t_oss
    model['TRANSZ'] = 13608.5 + 671.1 * sin_el - 286.7 * cos_el - 13.1 * t_oss
    model['TILTX'] = 236.7 - 66.1 * sin_el + 119.6 * cos_el + 0.0 * t_oss
    model['TILTY'] = 70.5 + 20.5 * sin_el + 72.0 * cos_el + 0.0 * t_oss
    return model[key]

def f9_model(key, el, t_oss):
    model = {}
    sin_el = np.sin(el * u.deg)
    cos_el = np.cos(el * u.deg)
    model['TRANSX'] = 2097.8 - 628.2 * sin_el - 1147.6 * cos_el + 0.0 * t_oss
    model['TRANSY'] = -919.7 + 199.7 * sin_el + 1046.9 * cos_el + 0.0 * t_oss
    model['TRANSZ'] = 293.4 + 725.2 * sin_el + 81.3 * cos_el - 46.7 * t_oss
    model['TILTX'] = 78.5 - 45.8 * sin_el + 48.3 * cos_el + 0.0 * t_oss
    model['TILTY'] = -83.3 + 56.6 * sin_el + 157.4 * cos_el + 0.0 * t_oss
    return model[key]

def plot_m2(t, xaxis='EL', model=None):
    x = t[xaxis]
    fig, axes = plt.subplots(3, sharex=True, figsize=(6, 9))
    pred = {}
    for k in ['TRANSX', 'TILTY', 'TRANSY', 'TILTX', 'TRANSZ']:
        if model is not None:
            pred[k] = model(k, t['EL'], t['OSSTEMP'])
        else:
            pred[k] = 0.0
    for a in axes:
        a.xaxis.grid()
    axes[0].scatter(x, t['TRANSZ']-pred['TRANSZ'], color='r')
    axes[0].set_ylabel("Focus (um)")
    axes[1].scatter(x, t['TRANSY']-pred['TRANSY'], color='r')
    axes[1].set_ylabel("Y Translation (um)", color='r')
    ax1_twin = axes[1].twinx()
    ax1_twin.scatter(x, t['TILTX']-pred['TILTX'], color='b')
    ax1_twin.set_ylabel("X Tilt (arcsec)", color='b', rotation=270, labelpad=20)
    axes[2].scatter(x, t['TRANSX']-pred['TRANSX'], color='r')
    axes[2].set_ylabel("X Translation (um)", color='r')
    axes[2].set_xlim(20, 90)
    axes[2].set_xlabel("Elevation (deg)")
    ax2_twin = axes[2].twinx()
    ax2_twin.scatter(x, t['TILTY']-pred['TILTY'], color='b')
    ax2_twin.set_ylabel("Y Tilt (arcsec)", color='b', rotation=270, labelpad=20)
    fig.tight_layout()
    fig.subplots_adjust(hspace=0)
    plt.show()

In [3]:
%cd /Volumes/Seagate2TB/wfsdat

/Volumes/Seagate2TB/wfsdat


In [4]:
with open("bino_2018.txt") as f:
    bino_files = f.readlines()
with open("f5_2018.txt") as f:
    f5_files = f.readlines()
with open("f9wfs_2018.txt") as f:
    f9_files = f.readlines()

In [5]:
keys = []
for k in HEADER_MAP:
    keys.append(HEADER_MAP[k]['fitskey'])

In [6]:
rows = []
for f in f9_files:
    p = Path(f.strip())
    zv = ZernikeVector()
    zv.load(str(p))
    if zv.rms < 1000.0 * u.nm:
        fitsfile = str(p.parent) + "/" + p.stem
        row = [fitsfile]
        hinfo = get_header_info(keys, fitsfile)
        row += hinfo
        rows.append(row)

In [7]:
f9_table = Table(rows=rows, names=["filename"] + keys)

In [8]:
rows = []
for f in bino_files:
    p = Path(f.strip())
    zv = ZernikeVector()
    zv.load(str(p))
    if zv.rms < 500.0 * u.nm:
        fitsfile = str(p.parent) + "/" + p.stem
        row = [fitsfile]
        try:
            hinfo = get_header_info(keys, fitsfile)
            row += hinfo
            rows.append(row)
        except:
            pass

In [9]:
bino_table = Table(rows=rows, names=["filename"] + keys)
len(bino_table), len(f9_table)

(1079, 254)

In [43]:
plt.scatter(bino_table['EL'], bino_table['TILTY'], color='r')
plt.show()

FigureCanvasNbAgg()

In [20]:
transx_resid = bino_table['TRANSX'] - bino_model('TRANSX', bino_table['EL'], 0.0)
tilty_resid = bino_table['TILTY'] - bino_model('TILTY', bino_table['EL'], 0.0)

In [21]:
az = bino_table['AZ']
az[az < 0] += 360.
plt.scatter(az, transx_resid)
plt.show()

FigureCanvasNbAgg()

In [22]:
plt.scatter(az, tilty_resid)
plt.show()

FigureCanvasNbAgg()

In [26]:
plt.scatter(bino_table['MJD'], az)
plt.show()

FigureCanvasNbAgg()

In [27]:
f9_transx_resid = f9_table['TRANSX'] - f9_model('TRANSX', f9_table['EL'], 0.0)
f9_tilty_resid = f9_table['TILTY'] - f9_model('TILTY', f9_table['EL'], 0.0)

In [28]:
f9_az = f9_table['AZ']
f9_az[f9_az < 0] += 360.
plt.scatter(f9_az, f9_transx_resid)
plt.show()

FigureCanvasNbAgg()

In [38]:
plot_m2(bino_table)

FigureCanvasNbAgg()

In [37]:
plt.close('all')